# Building a Restuarant chatbot that can converse in both Pigin English, Igbo and normal English
## Overview
A restuarant "Austa Kitchen" has an online store but doesn't have time to reply messages on the website, with this tool. They can focus on cooking while The chatbot helps reply customers.

## Method
I will use Open ai, claude and othe models to enhance responses. The prompts will be given to the system and additional information will be given to further enhance the capability.

This chatbot will be trained to be able to recommend other nigerian restuarants and website for more information

In [225]:
import gradio as gr
import openai as OpenAI
from dotenv import load_dotenv
import os
import anthropic
import json

In [226]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI

OpenAI API Key exists and begins sk-proj-


**The system_message is to customize the AI to reply as you.**

In [243]:
system_message = "You are restuarant chatbot and a customer service, a Nigerian Chef and instructor. " + \
                "You should respond truthfully. " + \
                "Always respond in pidgin english, english or Igbo Only. " + \
                "when asked about how much an item cost, it relates to cooking so respond"+\
                "You are nice and polite." +\
                "When asked about your name, always say Austa Kitchen" +\
                "when the user says hi YOU MUST REPLY with I am Austa, owner of Austa Kitchen and say something nice" +\
                "Always ask for review at the end of the customer order" +\
                "You are also to give a price order summary and list the menu"+\
                "Always reply politely and humourously"+\
                    "when asked about another question other than Austa kitchen and food RESPOND with 'Please make we talk the real matter way de ground'" +\
                "When asked what is your menu, STRICTLY reply with Jollof rice, eba and soup, fried rice, fish and chicken"

In [244]:
def chat(message, history):
        messages = [
                {"role":"system", "content": system_message}] + history + [{"role": "user", "content": message}]
        
        response = openai.chat.completions.create(
            model = "gpt-4o-mini", messages=messages)
            
        return response.choices[0].message.content

## Testing the system

In [245]:
gr.ChatInterface(fn=chat, 
                 type="messages",
                 title ="Kitchen Assistant Chatbot").launch(share= True)

* Running on local URL:  http://127.0.0.1:7909
* Running on public URL: https://8af1286dfa91191117.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [246]:
# menu = {"Fried Rice": "500 naira", "beans":"200 naira", "Jollof Rice" : "50 naira",
#          "7up": "300 naira", "Eba and soup": "200 naira", "Fish" : "500 naira", 
#          "Chicken" : "700 naira"}

## Creating a list for the menu so the LLM can be able to locate and use it

In [248]:
# items = {"Egusi": "500 naira", "beans": "200 naira"}
menu = {"fried rice": "500 naira", "beans":"200 naira", "jollof rice" : "50 naira",
         "efo riro": "300 naira", "eba and soup": "200 naira", "fish" : "500 naira", 
         "chicken" : "700 naira"}

#create a function to call it
def get_item_price(item_name):
    print(f"Tool get_item_price called for {item_name}") 
    item = item_name.lower() #convert item name to lower case
    return menu.get(item, "Unknown")

In [249]:
get_item_price("Jollof rice")

Tool get_item_price called for Jollof rice


'50 naira'

In [250]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_item_price",
    "description": "Get the price of a food item in the menu. Call this whenever you need to know the price for an item, for example when a customer asks 'How much does a beans cost'. When asked to list the menu, list the items here",
    "parameters": {
        "type": "object",
        "properties": {
            "item": {
                "type": "string",
                "description": "The menu the customer may be interested in buying",
            },
        },
        "required": ["item"],
        "additionalProperties": False
    }
}

In [251]:
tools = [{"type": "function", "function": price_function}]

In [252]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, item = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [253]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    item = arguments.get('item')
    price = get_item_price(item)
    response = {
        "role": "tool",
        "content": json.dumps({"item": item,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, item

In [254]:
# def handle_tool_call(message):
#     """
#     Handles multiple tool calls from an assistant message, executes the corresponding functions,
#     and returns a list of tool response messages.

#     Args:
#         message (Message): The assistant message containing the tool calls.

#     Returns:
#         list: A list of tool response messages (dictionaries).
#                Each tool response message contains role, content, and tool_call_id.
#     """
#     if not message.tool_calls:
#         raise ValueError("Message does not contain any tool calls.")

#     tool_outputs = []

#     for tool_call in message.tool_calls:
#         tool_call_id = tool_call.id

#         try:
#             arguments = json.loads(tool_call.function.arguments)
#         except json.JSONDecodeError as e:
#             tool_outputs.append({
#                 "role": "tool",
#                 "content": json.dumps({"error": f"Invalid JSON in tool call arguments: {e.msg}"}),
#                 "tool_call_id": tool_call_id
#             })
#             continue  # Move to the next tool call

#         item = arguments.get('item')

#         if not item:
#             tool_outputs.append({
#                 "role": "tool",
#                 "content": json.dumps({"error": "Item name not provided in tool call arguments."}),
#                 "tool_call_id": tool_call_id
#             })
#             continue  # Move to the next tool call

#         function_name = tool_call.function.name

#         if function_name == "get_item_price":
#             price = get_item_price(item)
#             response_content = json.dumps({"item": item, "price": price})
#         else:
#             response_content = json.dumps({"error": f"Tool function '{function_name}' not found."})

#         tool_outputs.append({
#             "role": "tool",
#             "content": response_content,
#             "tool_call_id": tool_call_id
#         })

#     return tool_outputs

In [255]:
#gr.ChatInterface(fn=chat, type="messages").launch()

gr.ChatInterface(
    fn=chat, 
    type="messages",
    title="Welcome to Austa Kitchen",          # Title shown at the top of the interface
    description="Ask me anything about Austa Kitchen!"  # Label shown below the title
).launch(share = True)

* Running on local URL:  http://127.0.0.1:7910
* Running on public URL: https://e3918693903e4b3274.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
